## Block Model SDK Examples

This notebook demonstrates how to work with Evo Block Models using the **Block Model SDK**, which provides a high-level, Pythonic interface for block model operations.

### SDK vs API Approach

This notebook uses the **Block Model SDK** (`evo.blockmodels.BlockModelAPIClient`) which:
- ✅ Provides simplified, high-level methods for common operations
- ✅ Handles complex API interactions behind the scenes
- ✅ Offers better error handling and validation
- ✅ Works directly with PyArrow tables and pandas DataFrames
- ✅ Recommended for most use cases

### Want to Learn More About the Underlying API?

If you're interested in understanding the raw Block Model API calls and more granular control, check out the `api-examples.ipynb` notebook in this same directory. The API examples show:
- Direct HTTP API calls to the Block Model service
- Lower-level control over requests and responses
- Detailed API response structures
- Advanced customization options

In [ ]:
from datetime import datetime

from evo.blockmodels import BlockModelAPIClient
from evo.notebooks import ServiceManagerWidget

cache_location = "./notebook-data"
input_path = f"{cache_location}/input"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

manager = await ServiceManagerWidget.with_auth_code(
    discovery_url="https://discover.api.seequent.com",
    redirect_url=redirect_url,
    client_id=client_id,
    cache_location=cache_location,
).login()

### Prepare Evo SDK parameters

In [ ]:
# Get the environment and connector from the ServiceManagerWidget instance.
# The environment contains the hub URL, organization ID, and workspace ID.
# The connector is used to make API calls to the Evo service.
environment = manager.get_environment()
connector = manager.get_connector()

service_client = BlockModelAPIClient(environment, connector, manager.cache)

### Demo 1: Create a regular block model

In [ ]:
import pyarrow
import pyarrow.parquet

from evo.blockmodels.data import RegularGridDefinition
from evo.blockmodels.endpoints.models import RotationAxis

name = f"Regular block model {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
description = "This is a description of my regular block model."
object_path = "/Jupyter/SDK-demo"  # Similar to a folder path.
coordinate_reference_system = "EPSG:3395"  # BlockSync only supports EPSG codes that are metre-based.
size_unit_id = "m"

grid_definition = RegularGridDefinition(
    model_origin=[1478500, 5174500, 100],
    rotations=[(RotationAxis.x, 0), (RotationAxis.y, 0), (RotationAxis.z, 0)],  # Null rotation for demo purposes
    n_blocks=[48, 68, 40],
    block_size=[25, 25, 25],
)

# Load data from parquet file as initial_data
initial_data = pyarrow.parquet.read_table("sample/data.parquet")

# Drop the dx, dy, dz columns from the initial_data
initial_data = initial_data.drop(["dx", "dy", "dz"])

block_model, version = await service_client.create_block_model(
    name=name,
    description=description,
    grid_definition=grid_definition,
    object_path=object_path,
    coordinate_reference_system=coordinate_reference_system,
    size_unit_id=size_unit_id,
    initial_data=initial_data,
)

print(f"Created block model with ID: {block_model.id}")

### Demo 2: Add new columns to a block model

In [ ]:
new_cols = pyarrow.table(
    {
        "x": [1478512.5, 1479687.5, 1479612.5],
        "y": [5174512.5, 5175087.5, 5175137.5],
        "z": [112.5, 337.5, 587.5],
        "Ag": [120, 155, 72],
        "Pt": [4.5, 5.3, 2.1],
    },
    schema=pyarrow.schema(
        {
            "x": pyarrow.float64(),
            "y": pyarrow.float64(),
            "z": pyarrow.float64(),
            "Ag": pyarrow.int32(),
            "Pt": pyarrow.float64(),
        }
    ),
)

# Add new columns to the block model and include units for the "Pt" column.
version_with_new_columns = await service_client.add_new_columns(
    bm_id=block_model.id,
    data=new_cols,
    units={"Pt": "g/t"},
)

print(f"Updated block model with new columns: {[col for col in new_cols.column_names if col not in ['x', 'y', 'z']]}")

### Demo 3: Query the latest version of a block model

In [ ]:
# Table will be a Pyarrow Table
table = await service_client.query_block_model_as_table(
    bm_id=block_model.id,
    columns=["*"],  # Include all columns by using the * wildcard
)

# Note: The column UUID is returned instead of the column name.
display(table.to_pandas())

### Demo 4: Query a block model using a bounding box

In [ ]:
from evo.blockmodels.endpoints.models import BBoxXYZ, FloatRange

# Select a bounding box to query
bounding_box = BBoxXYZ(
    x_minmax=FloatRange(min=1478587.5, max=1478662.5),
    y_minmax=FloatRange(min=5174562.5, max=5174712.5),
    z_minmax=FloatRange(min=137.5, max=187.5),
)

# Table will be a Pyarrow Table
table = await service_client.query_block_model_as_table(bm_id=block_model.id, columns=["*"], bbox=bounding_box)

# Note: The column UUID is returned instead of the column name.
display(table.to_pandas())

### More to come...

New examples will be added in the future as the Block Model SDK continues to evolve!

In the meantime, check out the `api-examples` notebook to see what else the Block Model API is capable of.